In [1]:
from IPython.display import Audio
import numpy as np
import mido
import pandas as pd
import matplotlib.pyplot as plt
import pyaudio as pa
from threading import Thread

In [2]:
# Clase abstracta Instrument para centralizar el comportamiento
class Instrument:
    
    # Frecuencia fundamental de cada nota
    notes = {'A' : 27.5, 'A#' : 29.0, 'B' : 30.87, 'Bb' : 29.135, 'C' : 16.35, 'C#' : 17.32, 'D' : 18.35, 'D#' : 19.0, 'E' : 20.6, 'Eb' : 19.445, 'F' : 21.83, 'F#' : 23.12 ,'G' : 24.5, 'G#' : 25.96}

    # Conversiones de hexa a nota + octava.
    df = pd.read_csv('notes.csv').set_index('Num')
    
    # Constructor. Tira error si se intenta instanciar.
    def __init__(self, *args, **kwargs):
        raise Exception ('Cannot instantiate abstract class Instrument')
        
    # Carga de archivo MIDI
    def load(self, midi): 
        self.midi, point = midi, 0
        self.tempos = np.array([[0], [120]])
        
        # Genera vector de tempos para cada momento de la canción
        for ev in self.midi.tracks[0]:
            point += ev.time
            if ev.is_meta and ev.type == 'set_tempo':
                if point == 0: self.tempos = np.array([[point], [ev.tempo]])
                else: self.tempos = np.append(self.tempos, [[point], [ev.tempo]], axis = 1)

        # Genera vector de tiempo/tick para cada momento de la canción
        self.time_steps = self.tempos[1] / (1e6 * self.midi.ticks_per_beat)
    
    # Devuelve el tiempo/tick para cierto punto de la canción
    def current_timestep(self, point):
        return self.time_steps[self.tempos[0] <= point][-1]

    # Sintetiza el track elegido del MIDI previamente cargado.
    # add: Si es True, entonces no borra los datos anterior, sino que los superpone.
    # nochannels: Qué canales no cargar.
    # stretch_lim: Límite inferior de frecuencia para lingering
    # lowtone: Cuántos armónicos bajarle a las notas.
    def synthesize(self, track : int, fs : float, add = False, nochannels = [], stretch_lim = 150, lowtone = 0):
        self.track = self.midi.tracks[track]
        self.fs = fs

        if not hasattr(self, 'sound'): self.sound = np.zeros(int(np.ceil(self.fs * self.midi.length)))
        elif not add: self.sound[:] = 0
            
        point = real_time = 0
        self.used_off = []
        
        # Itera por cada evento en el track
        for i, ev in enumerate(self.track):
            
            # Carga los valores de tiempo (real y ticks)
            real_time += ev.time * self.current_timestep(point)
            point += ev.time
            
            # Si el evento es apto para ser cargado como nota...
            if not ev.is_meta and ev.type == 'note_on' and not ev.channel in nochannels and not i in self.used_off:
                
                # Obtiene información de la nota (frecuencia + octava)
                info = self.df.loc[ev.note]
                fnote, octave = self.notes[info['Note']], info['Octave']
            
                # Busca cuándo termina la nota y obtiene su duración
                d, v = self._find_off(ev.note, i, ev.channel)
                duration = d * self.current_timestep(point) * 1.5
                
                # Agrega la nota al vector de sonido
                if duration: self._add_note(fnote, octave, duration, max(v, ev.velocity), real_time, stretch_lim, lowtone)
    
    # Agrega una nota al vector de sonido
    # fnote: Frecuencia
    # octave: Octava
    # duration: Duración
    # velocity: Intensidad
    # stretch_lim: Límite inferior de frecuencia para lingering
    # lowtone: Cuántos armónicos bajarle a las notas.
    def _add_note(self, fnote, octave, duration, velocity, real_time, stretch_lim, lowtone):
        
        # Genera la nota y normaliza su amplitud
        nt = self._gen_note(freq = fnote * 2**max(0, octave-lowtone), dur = duration, stretch_lim = stretch_lim)
        nt *= velocity / np.abs(nt).max() * octave
        
        # Busca el índice en el que empieza la nota
        idx = int(np.round((self.fs * real_time), 0))

        # Agrega puntos en caso de sobrar por redondeos
        if idx + nt.size > self.sound.size:
            self.sound = np.append(self.sound, np.zeros(idx + nt.size - self.sound.size))
                    
        # Agrega la nota
        self.sound[idx : idx + nt.size] += nt
    
    # Busca cuándo termina una nota
    # note: La nota que debe terminar
    # idx: Índice a partir del cual puede estar el evento de off
    # channel: Canal al cual corresponde la nota
    def _find_off(self, note, idx, channel):
        tot = 0
        # Itera por cada evento, sumando los tiempos
        for i, ev in enumerate(self.track[idx + 1:]):
            tot += ev.time
            
            # Si encontró el correcto, devuelve la duración y la intensidad
            if not ev.is_meta and ev.type in ['note_on', 'note_off'] and ev.note == note and ev.channel == channel:
                self.used_off.append(i + idx + 1)
                return tot, ev.velocity
        return 0#raise Exception('MIDI error - Note never turned off')

In [3]:
# Clase abstracta KSInstrument para los instrumentos sintetizados por Karplus-Strong
class KSInstrument(Instrument):
    
    # Constructor. Tira error si se intenta instanciar.
    def __init__(self, *args, **kwargs):
        raise Exception('Cannot instantiate abstract class KSInstrument')
        
    # Método que se debe sobreescribir para cada clase de instrumento. Genera una nota.
    # freq: Frecuencia de la nota
    # dur: Duración de la nota
    # stretch_factor: Límite inferior de frecuencia para lingering
    # b: Parámetro propio de cada instrumento
    # La frecuencia de sampleo fs es propia de la instancia, y se obtiene con self.fs   
    def _gen_note(self, freq, dur, stretch_factor, b):
        
        # Genera el N (o L) de Karplus-Strong para la frecuencia de la nota y fs
        N = int(np.round(self.fs/freq - 1/(2 * stretch_factor), 0))
        
        # Crea vector resultante con la duración correspondiente
        samples = np.zeros(int(np.round(self.fs * dur, 0)))
        
        # Genera ruido inicial con -1 y 1.
        samples[ : N] = (2 * np.random.randint(0, 2, N) - 1).astype(float)
        
        k = np.zeros(N)
        r = np.random.binomial(1, 1 / stretch_factor, samples.size).astype(bool)
        
        for i in range(N, N * (1 + samples.size//N), N):
            idx = r[i : i + N]
            k = k[ : idx.size]
            
            t1 = samples[i - N : i - N + k.size]
            if i == N: t2 = np.concatenate(([samples[i - N - 1]], samples[i - N : i - N - 1 + k.size]))
            else: t2 =  samples[i - N - 1 : i - N - 1 + k.size]
                
            k[~idx] = t1[~idx]
            k[idx] = (t1 + t2)[idx]/2
            
            samples[i : i + N] = b * k
            
        return samples
    
# Clase guitarra, hereda de KSInstrument. Sobreescribe a _gen_note con b = 1
class Guitar(KSInstrument):
    
    def __init__(self, *args, **kwargs):
        pass
        
    def _gen_note(self, freq, dur = 1, b = 1, stretch_lim = 150):
        return super()._gen_note(freq = freq, dur = dur, b = 1, stretch_factor = max(1, freq / stretch_lim))
    
# Clase harpa, hereda de KSInstrument. Sobreescribe a _gen_note con b = -1
class Harp(KSInstrument):
    
    def __init__(self, *args, **kwargs):
        pass
        
    def _gen_note(self, freq, dur = 1, velocity = 50, b = 1, stretch_lim = 150):
        return super()._gen_note(freq = freq, dur = dur, b = -1, stretch_factor = max(1, freq / stretch_lim))
    
class drumB:
    def __init__(self):
        pass
    def __mul__(self, other):
        return other * (-1)**np.random.binomial(1, .5, other.size)

# Clase tambor, hereda de KSInstrument. Sobreescribe a _gen_note con b = drumB(), que al multiplicar toma valor 
# 1 o -1 con probabilidad 0.5.
class Drum(KSInstrument):
    def __init__(self, *args, **kwargs):
        pass
    
    def _gen_note(self, freq, dur = 1, velocity = 50, b = 1, stretch_lim = 150):
        return super()._gen_note(freq = freq, dur = dur, b = drumB(), stretch_factor = max(1, freq / stretch_lim))

In [4]:
# Clase reproductor. 
class Player:
    
    CHUNKSIZE = 1024
    working = False
    
    # Constructor. Si no tiene PyAudio, lo crea. Setea los parámetros iniciales.
    def __init__(self, *args, **kwargs):
        if not Player.working: 
            Player.pAudio = pa.PyAudio() 
            Player.working = True
        
        self.paused = self.stop_playing = False
        self.th = Thread(target = self._keep_playing)
    
    # Carga de nuevo sonido
    def load(self, data, fs):
        
        # Normaliza el sonido y actualiza los parámetros
        self.sound = (data / np.abs(data).max()).astype(np.float32)
        self.fs = fs
        self._close_stream()
        
        # Crea nuevo stream
        self.stream = self.pAudio.open(rate = self.fs, format = pa.paFloat32, channels = 1, output = True)
        self.stream.start_stream()
        
    # Thread de loopeo. Va tomando datos de a CHUNKSIZE y mandándolos al stream
    def _keep_playing(self):
        data = self.sound[ : self.CHUNKSIZE].tobytes()
        self.idx += 1
        
        while len(data) and not self.stop_playing:
            if not self.paused:
                self.stream.write(data)
                data = self.sound[self.idx * self.CHUNKSIZE : (self.idx + 1) * self.CHUNKSIZE].tobytes()
                self.idx += 1

        self.stop_playing = False
        
    # Play. Inicia nueva reproducción o saca la pausa, dependiendo del estado actual
    def play(self, **kwargs): 
        
        if self.paused: 
            self.paused = False
        
        else:
            self.stop_playing = True
            if self.th.is_alive(): self.th.join() 

            self.idx, self.stop_playing = 0, False
            self.th = Thread(target = self._keep_playing)
            self.th.start()
    
    # Cierra el stream previo
    def _close_stream(self):
        self.stop_playing = True
        if self.th.is_alive(): self.th.join()
            
        if hasattr(self, 'stream') and self.stream.is_active(): 
            self.stream.stop_stream()
            self.stream.close()   
    
    # Termina la reproducción
    def stop(self): 
        self.stop_playing = True
        self.paused = False
    
    # Pausa la reproducción
    def pause(self): 
        self.paused = True
        
    # Libera recursos de PyAudio. Llamar siempre antes de cerrar el programa.
    def close(self): 
        self._close_stream()
        self.pAudio.terminate()
        Player.working = False

In [5]:
try: player.close()
except: pass
player = Player()
guitar = Guitar()
harp = Harp()

## Hotel California

In [12]:
file = mido.MidiFile("Samples/hotelcal2.mid")
guitar.load(midi = file)
guitar.synthesize(fs = 48000, track = 0, lowtone = 0, stretch_lim = 500)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Under the Sea

In [ ]:
file2 = mido.MidiFile("Samples/Disney_Themes_-_Under_The_Sea.mid")
guitar.load(midi = file2)
harp.load(midi = file2)
for i in range(1, len(file2.tracks)):
    print(f'Track {i} of {len(file2.tracks) - 1}')
    if not i%2: guitar.synthesize(fs = 2**16, track = i, add = i != 2)
    else: harp.synthesize(fs = 2**16, track = i, add = i != 1)

if guitar.sound.size < harp.sound.size: guitar.sound = np.append(guitar.sound, np.zeros(harp.sound.size - guitar.sound.size))
else: harp.sound = np.append(harp.sound, np.zeros(guitar.sound.size - harp.sound.size))
player.load(guitar.sound + harp.sound, guitar.fs)
player.play()

## Never Gonna Give You Up

In [ ]:
file3 = mido.MidiFile("Samples/Never-Gonna-Give-You-Up-3.mid")
guitar.load(midi = file3)
guitar.synthesize(fs = 48000, track = 0)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

## Happy Birthday

In [ ]:
file4 = mido.MidiFile("Samples/happy_birthday.mid")
guitar.load(midi = file4)
guitar.synthesize(fs = 48000, track = 1)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

## Losing My Religion

In [26]:
file5 = mido.MidiFile("Samples/losing_my_religion.mid")
guitar.load(midi = file5)
guitar.synthesize(fs = 48000, track = 2, lowtone = 1, stretch_lim = 500)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Pirates of the Caribbean

In [8]:
file8 = mido.MidiFile("Samples/26593_Hes-a-Pirate.mid")
guitar.load(midi = file8)
guitar.synthesize(fs = 48000, track = 1, stretch_lim = 500, lowtone = 0)
guitar.synthesize(fs = 48000, track = 2, stretch_lim = 500, lowtone = 0, add = True)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Game of Thrones

In [14]:
file10 = mido.MidiFile("Samples/GameofThrones.mid")
guitar.load(midi = file10)
guitar.synthesize(fs = 48000, track = 1, stretch_lim = 300)
guitar.synthesize(fs = 48000, track = 2, stretch_lim = 500, add = True)
guitar.synthesize(fs = 48000, track = 3, stretch_lim = 500, add = True)
guitar.synthesize(fs = 48000, track = 4, stretch_lim = 500, add = True)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Moonlight Sonata

In [20]:
file11 = mido.MidiFile("Samples/Beethoven-Moonlight-Sonata.mid")
guitar.load(midi = file11)
guitar.synthesize(track = 1, fs = 48000, lowtone = 1)
guitar.synthesize(track = 2, fs = 48000, add = True)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Simpsons

In [10]:
file12 = mido.MidiFile("Samples/Simpsons.mid")
guitar.load(midi = file12)
for i, track in enumerate(file12.tracks):
    guitar.synthesize(fs = 48000, track = i, add = i != 0)
    
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


## Tetris

In [18]:
file13 = mido.MidiFile("Samples/Tetris_-_Theme_A.mid")
guitar.load(midi = file13)
for i in range(1, len(file13.tracks)):
    guitar.synthesize(fs = 48000, track = i, add = i!=1)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


In [13]:
player.pause()

In [19]:
player.stop()